In [1]:
pwd

'/Users/dharamvir/Desktop'

In [2]:
pip install pydicom

Note: you may need to restart the kernel to use updated packages.


In [21]:
import numpy as np
import pandas as pd
from skimage.io import imread
import seaborn as sns
import matplotlib.pyplot as plt
import pydicom as dicom
import os
import scipy.ndimage


from skimage import measure, morphology
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

In [22]:
path = 'Pancreas/'
patients = os.listdir(path)
patients.sort()

In [34]:
print("Number of DICOM files:", len(os.listdir(path)))

Number of DICOM files: 130


In [35]:
dicom_data = process_data('path/*.dcm')

NameError: name 'process_data' is not defined

In [30]:
def load_scan(path):
    slices = [dicom.read_file(path + '/' + s) for s in os.listdir(path)]
    slices.sort(key = lambda x: float(x.ImagePositionPatient[2]))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

In [31]:
def get_pixels_hu(slices):
    image = np.stack([s.pixel_array for s in slices])

    image = image.astype(np.int16)
    image[image == -2000] = 0

In [32]:
first_patient = load_scan(path)
first_patient_pixels = get_pixels_hu(first_patient)

In [36]:
def return_dcm(path):
    out_dcm = {}
    for dirName, subdirList, fileList in os.walk(file_path):
        c_dcm = []
        cur_name = ""
        dir_split = dirName.split("/")
        for f_chk in dir_split:
            if check_term in f_chk:
                cur_name = f_chk
        for filename in fileList:
            if ".dcm" in filename.lower():
                name = int(os.path.splitext(filename)[0])
                c_dcm.append((os.path.join(dirName,filename), name))
        if len(c_dcm) > 0:
            c_dcm = sorted(c_dcm, key = lambda t: t[1]) 
            out_dcm[cur_name] = [c[0] for c in c_dcm] 
    return out_dcm

In [ ]:
def plot_slice(slice_in, is_anns = False, num_anns = 4):
    
    slice_in = np.squeeze(slice_in)
    plt.figure()
    plt.set_cmap(plt.bone())
    if is_anns:
        plt.pcolormesh(slice_in, vmin = 0, vmax = num_anns - 1)
    else:
        plt.pcolormesh(slice_in)
    plt.show()